In [ ]:

# %pip install fast-plate-ocr
# %pip install onnxruntime% 
# %pip install --upgrade ultralytics
# %pip install pillow pillow-heif


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\ajlee\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# import easyocr  # Changed for using LicensePlateRecognizer OCR
import pandas as pd
import os
import cv2
from ultralytics import YOLO
from pathlib import Path  

# Directory of files
input_dir = r"C:\Users\ajlee\Documents\LicensePlateProject\License Plate Photos" # Where image files go
output_dir = r"C:\Users\ajlee\Documents\LicensePlateProject\Processed Images Output" # Output of annotated images that have been blurred, cropped etc.
weights_path = r"C:\Users\ajlee\Documents\LicensePlateProject\Best_Yolov8_Trained_Model\weights\last.pt" # Location of the yolo model to be used
csv_path = r"C:\Users\ajlee\Documents\LicensePlateProject" # The location of a .csv file containing all the license plate strings collected from the program

# **Preprocess_Plate Function**
The purpose of this function is to pre-process images so that when they enter the OCR pipeline they can be easily interpreted. OCR has better recognition when we remove color, blur background noise, and make the letters contrast from the background.

## **Step 1: Gray Scale**     
`gray   = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)` -> This command takes the `img` a numeric array representing the mathamatical arrangement of the cropped license plate image and transforms it from *OpenCV's* default BGR Color Setting to a gray scale, it collapeses the 3 channels of Blue, Green, and Red into 1 Channel Gray.    

The mathematical formula for Gray = 0.299·R + 0.587·G + 0.114·B 

---

## **Step 2: Bilateral Blurring (Noise Removal)**    
`blur = cv2.bilateralFilter(gray, d, sigmaColor, sigmaSpace)` -> Bilateral Blur allows us to blur only specific parts of an image, in this example it helps us avoid blurring important parts like the sharp edges of a character, necessary in identifying letters / numbers on the plate. A standard Gaussian Blur, blurs the entire picture including the license plate characters making it difficult for the OCR program to covert the image to text. Gaussian Blur is based on distance from the center point, and treats all pixels as the same, therefore it can cause edges of characters to soften. Bilateral Filter adds the capability to check how similar nearby pixels are, based upon gray-scale values and apply a weighted blur to pixels that aren't as similar, allowing us to keep sharp edges of characters.    

![alt text](bilateral_demo.png)

### **Setting The Values**    
**d** -> Size of the surrounding pixels to be blurred, measured in diameter of pixels, a larger number refers to a larger area to be groupped     
`d = 5-15` for 720p images,    
`d = 25+` if high-res and noisy    

**sigmaColor** -> Tells the command how much difference in the shade of gray to observe or ignore in neighboring pixels      
`sigmaColor = 5` means that it should notice every difference in shade coloring, and to be highly selective in blurring      
`sigmaColor = 17` is a more general blur and will consider more differences in the gray shades as similar to each other, helps blur out noise while keeping edges   sharp      
`sigmaColor = 50` treat very different shades of gray as the same, applying a greater blur to surrounding areas, leading to more blur and potentially less edge preservation     
   
**sigmaSpace** -> Tells how many pixels in a specified size / area are considedered and compared accordingly     
`sigmaSpace = 5` is very restrictive area to be considered / compared (good for tiny crops 100 x 40 px plates/width)    
`sigmaSpace = 17` is the middle groud to hopefully smooth the letter strokes without crossing over too much (500 - 1000 px-wide plates)    
`sigmaSpace = 50` Blur could creep over into other letters and cause splotching of the blur (doubling plate width -> double `sigmaSpace`)    

---

## **Step 3: CLACHE (Equalize Brightness Within The Photo)**    
```
clahe  = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
boost  = clahe.apply(blur)
```     
![alt text](clahe_demo.png)
-> The purpose of this is to fix any lighting issues from the image where half of the plate was shot in the sun and is bright and the other half could be dark in a shadow.     

### **What Happens When We Apply CLACHE: Contrast-Limited-Adaptive Histogram Equalization**       
What typically happens is that if we took an image that had been gray-scaled and looked at a histogram of all the entire range from black to white with all the pixels in an image we would notice that the histogram would be in the center of the scale meaning the image has low contrast and therefore makes it harder for the OCR to read. The goal of this step is to equalize the histogram and ensure all the shades from black to white are being utilized therefore giving the image greater contrast and readability.    

![alt text](grayscalehistogrambefore.png)
![alt text](grayscalehistogramafter.png)
![alt text](originalvs.equalized.png)

### **How It Works**       
Step 1. **Tiling** -> This works by breaking down the image into smaller squares, in this case 8 x 8 pixel squares which the brightness of pixels are calculated into a histogram to create a brightness curve.    

Step 2. **Histogram Equilization** -> Re-maps each square set of pixels where the darkest pixel in the 8 x 8 square is set to 0 (Black) and the lightest pixele is set to 255 (White) and the rest of the pixels are then spread over the histogram to equalize it.

Step 3. **clipLimit** sets the maximum height any histogram bar can grow ensuring that the contrast is blown out of proportion. Any bins of the histogram that hit the **clipLimit** the remaining height of that bin is equally distribuited among the other bins this is to ensure that patches of the images containg similar pixels aren't blown into proportion causing a hole / splotch in the image.    

![alt text](effectofcliplimit.png)

### **Setting The Values**
| Python arg     | Typical range     | Effect                                                                                          | Tune it when…                                                                           |
| -------------- | ----------------- | ----------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------- |
| `tileGridSize` | (8, 8) … (16, 16) | Smaller tiles → finer adaptation, but risk noise boost. Larger tiles → smoother, more “global.” | Plates under strong spotlights → shrink to (4, 4). Grainy ISO noise → grow to (12, 12). |
| `clipLimit`    | 1.5 – 3.0         | Lower = softer boost (noise-safe). Higher = stronger pop (can over-amp salt-and-pepper).        | If letters look washed out → ↑ clipLimit. If background speckles glow → ↓ clipLimit.    |

      
---

## **Step 4: Adaptive Thresholding (Distingushing Characters From a Gray-Scale to Binary Black and White)**
```
thresh = cv2.adaptiveThreshold(
boost, 255,
cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
cv2.THRESH_BINARY_INV,   # white chars / dark background works well
19,9 
)
```
-> **Adaptive Thresholding** is a changing function that helps distinguish license plate characters from photos of varying brightness, darkness, distance and etc. The goal of this system is to highlight the exact outline of the numbers / letters of the license plate by setting the characters to white and the background to black. This creates a high contrast black and white image ensuring that the OCR only picks up the license plate number and nothing else.       
![alt text](adaptive_threshold_demo.png)    

### **How it Works**  
Step 1. **Setting a Threshold** -> The first step is to set a threshold value for example 128, any pixel darker(less) than 128 is set to black and any pixel lighter(greater) than 128 is set to white     
Step 2. **Block Size** -> Determines the size of the area that the pixels will be compared and averaged.      
Step 3. **Subtract a Constant** -> A constant value is subtracted from the local mean to help the cut-off distinguish dark and light values     
Step 4. **Compare the pixels** -> We compare the pixels value with our threshold and set them to either black or white only.

### **Setting The Values**
| Python argument  | What it **is / does**                                         | **Why** it matters                                                                                                         | **How to pick a value** (license-plate guidelines)                                                                                           |
| ---------------- | ------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------- |
| `src`            | Grayscale input image *(`uint8`)*                             | Must be a single channel (0-255) for thresholding math.                                                                    | Before calling, convert with `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)` and run your pre-processing (bilateral, CLAHE).                         |
| `maxValue`       | Value written to “white” pixels (usually **255**)             | Lets you create masks at any intensity, but 255 = conventional white.                                                      | Keep **255** unless your post-processing expects a different range.                                                                          |
| `adaptiveMethod` | **`cv2.ADAPTIVE_THRESH_MEAN_C`** or **`_GAUSSIAN_C`**         | Chooses how the local average is computed.                                                                                 | • Use **GAUSSIAN** when glare/hot-spots exist (weights centre more).<br>• Use **MEAN** when lighting varies smoothly (garage shadows).       |
| `thresholdType`  | **`cv2.THRESH_BINARY`** or **`..._INV`**                      | Sets whether bright or dark pixels become white.                                                                           | OCR pipelines usually want **letters as white blobs** ⇒ pick **`THRESH_BINARY_INV`**.                                                        |
| `blockSize`      | **Odd** window side length (e.g. 15, 19, 25)                  | Defines the neighbourhood whose average brightness becomes the local cut-off. Bigger = smoother, smaller = more sensitive. | **Rule of thumb** → `blockSize ≈ 3 × stroke-width`.<br>• 100 px-wide crop → 7–11<br>• 500 px-wide crop → 15–25<br>• ≥1000 px crop → 31–41    |
| `C`              | Constant **subtracted** from the local mean before comparison | Nudges the cut-off darker so letter strokes (usually darker than background) flip to white after inversion.                | Start with **`C ≈ ½ × stroke-width + 5`**.<br>‣ Letters look **hollow/thin** → *decrease C*.<br>‣ Letters look **fat/merge** → *increase C*. |     
       
---


## **Step 5: Morphological Close (Fill in Any Pin Holes Within The Characters)**
```
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
```
-> Adaptive thresholding can often leave hairline breaks in the characters, Morphological closing fills in these breaks, painting the holes within characters white and cleaning up edges / spillovers of the characters by painting them black

![alt text](morph_close_demo.png)

### **How it Works**    
**Dialation** -> Stretches the white pixels over to neighboring areas by a certain size to cover any breaks or holes that could be inside the character    
**Erosion** -> Cleans the edges of the characters by checking the position of each pixel and only keeping it white if all neighboring pixels are also white

### **Setting The Values**

| Python variable / argument                           | What it **is / does**                                                                 | **Why** it matters                                                                                 | **How to choose a good value** (license-plate tips)                                                                                                                                                                                                                                  |
| ---------------------------------------------------- | ------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `thresh` (first arg to `morphologyEx`)               | The binary image coming from adaptive thresholding (white letters, black background). | Closing operates only on white pixels, so the *quality* of this mask drives everything downstream. | Make sure you binarised with a sensible `blockSize` & `C`; otherwise closing can’t fix large holes.                                                                                                                                                                                  |
| `cv2.MORPH_CLOSE` (second arg)                       | Operation = **dilation → erosion** (in that order).                                   | Fills hairline gaps & tiny holes while keeping letter size roughly the same.                       | Keep as `MORPH_CLOSE`; switch to `MORPH_OPEN` only if you need to *delete* small isolated white specks.                                                                                                                                                                              |
| `kernel = cv2.getStructuringElement(shape, ksize)`   | Tiny “stamp” slid across the image to decide which neighbours are affected.           | Size and shape dictate how big a gap can be sealed and in which directions.                        | • **Shape**: `cv2.MORPH_RECT` (square) works for plate fonts; `MORPH_ELLIPSE` keeps corners rounder; `MORPH_CROSS` preserves diagonals.<br>• **Size (ksize)**: *(3, 3)* or *(5, 5)*.<br>  - 3×3 closes 1-px gaps (most common).<br>  - 5×5 closes 2-px gaps but may thicken strokes. |
| `iterations` (4th arg)                               | How many times to apply the close.                                                    | More iterations seal bigger gaps but thicken letters and risk merging adjacent glyphs.             | Start with **1**.<br>• Bump to **2** only if you *visually* see leftover breaks.<br>• Never go above 3 for normal plate crops.                                                                                                                                                       |
| *(optional)* `anchor` (defaults to centre of kernel) | Offset of the kernel’s origin.                                                        | Almost never changed for closing.                                                                  | Leave at default `(-1,-1)` so origin = centre.                                                                                                                                                                                                                                       |
| *(optional)* `borderType`, `borderValue`             | How OpenCV treats image edges.                                                        | Irrelevant for fully cropped plates.                                                               | Stick with default (`cv2.BORDER_CONSTANT`, value = 0).                                                                                                                                                                                                                               |


In [69]:
# Pre-processing of a license plate image to help improve OCR accuracy
def preprocess_plate(img):
    
    # Gray-scale removes color noise from image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur's the background while keeping the edges of the license plate characters sharp
    blur = cv2.bilateralFilter(
        src = gray, 
        d = 15, 
        sigmaColor = 25, 
        sigmaSpace = 15
        )

    # Equalize contrast of the image in case it's bright or dark in specific areas of the image
    clahe  = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    boost  = clahe.apply(blur)

    # Adaptive threshold turns the characters completely white and the background completey black
    thresh = cv2.adaptiveThreshold(
        boost, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,   # ensures that the characters turn white and the background is black
        blockSize = 29,
        C = 11
    )

    # Light morphological close to seal tiny gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)

    return cleaned

In [70]:


# Standardized image parameters
imgsize = 1920 # Standardized size to set all images before entering the YOLO model
class_conf = 0.32 # Only keeps YOLO detections with >= (X %) confidence score

# Post YOLO image size
STD_WIDTH = 600 # Standarized size that the image is set to after the YOLO model crops out the detected license plate

# Standardized percentage of the post-cropped image that will be removed to help eliminate noise such as license plate frames, state, registration dates etc.
TOP_TRIM_FRAC     = 0.0 # % removed from the top of the image
BOTTOM_TRIM_FRAC  = 0.0 # % removed from the bottom of the image
LEFT_TRIM_FRAC    = 0.0 # % removed from the left side of the image
RIGHT_TRIM_FRAC   = 0.0 # % removed from the right side of the image

# **Setting Up The YOLO Model / OCR Reader**
We define which YOLO model and OCR reader we want to use for this project. We also create an empty list to easily pull our OCR results.

### Line-By-Line Explanation of Code 

`model = YOLO(weights_path)` -> Sets which model to be used by YOLO to the *weights_path* folder 
`reader = LicensePlateRecognizer("global-plates-mobile-vit-v2-model")` -> Sets the OCR reader to be used
`ocr_results = []` -> Creates an empty list to be filled with dictionaries that will later be turned into a .csv file

In [71]:
from fast_plate_ocr import LicensePlateRecognizer
ocr = LicensePlateRecognizer("cct-s-v1-global-model") # License Plate Specific OCR

# Setup & load object detection trained model
model = YOLO(weights_path)
plate_class_name = "License_Plate" # Name of class objects as defined by the YAML file
# *reader = easyocr.Reader(['en'], gpu=False) # <- old OCR model (non-license plate specific OCR reader)
reader = LicensePlateRecognizer("global-plates-mobile-vit-v2-model") # OCR reader to be used
ocr_results = [] # collect rows for the CSV
os.makedirs(output_dir, exist_ok=True)
print(model.names)

{0: 'License_Plate'}


In [73]:
# Helper to read any image (JPG/PNG/HEIC) → OpenCV BGR array
from pathlib import Path
import numpy as np
import cv2
from PIL import Image
import pillow_heif

def read_image_any(path: str) -> np.ndarray | None:
    """
    Returns an image as a NumPy array in BGR (OpenCV) format, or None if unreadable.
    - JPG/PNG/etc.: read via cv2.imread
    - HEIC/HEIF: read via pillow-heif + Pillow, converted to BGR for OpenCV
    """
    ext = Path(path).suffix.lower()
    if ext in {".heic", ".heif"}:
        try:
            heif_file = pillow_heif.read_heif(path)
            img_pil = Image.frombytes(
                heif_file.mode, 
                heif_file.size, 
                heif_file.data, 
                "raw", 
                heif_file.mode, 
                heif_file.stride
            )
            img_rgb = np.array(img_pil)  # RGB
            if img_rgb.ndim == 2:
                return cv2.cvtColor(img_rgb, cv2.COLOR_GRAY2BGR)
            else:
                return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"[HEIC read error] {path} -> {e}")
            return None
    else:
        return cv2.imread(path, cv2.IMREAD_COLOR)


In [78]:
# ===== Robust processing loop for JPG/PNG/HEIC with YOLO + OCR =====
import os, sys, subprocess, traceback
from pathlib import Path
import numpy as np
import cv2

# --- ensure HEIC support if you are using .heic images ---
try:
    import pillow_heif
    from PIL import Image
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pillow", "pillow-heif"])
    import pillow_heif
    from PIL import Image

# --- universal reader: JPG/PNG via OpenCV, HEIC via pillow-heif ---
def read_image_any(path: str):
    ext = Path(path).suffix.lower()
    if ext in {".heic", ".heif"}:
        try:
            heif = pillow_heif.read_heif(path)
            img_pil = Image.frombytes(heif.mode, heif.size, heif.data, "raw", heif.mode, heif.stride)
            img_rgb = np.array(img_pil)
            if img_rgb.ndim == 2:
                return cv2.cvtColor(img_rgb, cv2.COLOR_GRAY2BGR)
            return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"[HEIC read error] {path} -> {e}")
            return None
    else:
        return cv2.imread(path, cv2.IMREAD_COLOR)

# --- defaults only used if not defined earlier in your notebook ---
try:
    STD_WIDTH
except NameError:
    STD_WIDTH = 640  # resize width for the plate crop (keeps aspect)

for _name, _default in [
    ("TOP_TRIM_FRAC", 0.05),
    ("BOTTOM_TRIM_FRAC", 0.05),
    ("LEFT_TRIM_FRAC", 0.03),
    ("RIGHT_TRIM_FRAC", 0.03),
]:
    if _name not in globals():
        globals()[_name] = _default

def _noop_preprocess(img):
    return img

if "preprocess_plate" not in globals():
    preprocess_plate = _noop_preprocess  # if you didn't define it earlier, do nothing

# --- gather images ---
os.makedirs(output_dir, exist_ok=True)
valid_exts = (".jpg", ".jpeg", ".png", ".heic", ".heif")

all_files = []
for root, _, files in os.walk(input_dir):
    for fname in files:
        if fname.lower().endswith(valid_exts):
            all_files.append(os.path.join(root, fname))

print(f"Found {len(all_files)} image(s).")
print("Preview:", [Path(p).name for p in all_files[:5]])

# --- process images ---
processed = 0
failed = []

for i, img_path in enumerate(all_files, start=1):
    try:
        img = read_image_any(img_path)
        if img is None:
            print(f"Unreadable: {img_path}")
            continue

        # run detector
        results = model(img)  # <<< you need this before iterating results

        # where to save this image's outputs
        rel = os.path.relpath(img_path, input_dir)
        rel_dir = os.path.dirname(rel)
        save_root = os.path.join(output_dir, rel_dir)
        os.makedirs(save_root, exist_ok=True)

        fname = Path(img_path).name
        plate_idx = 0  # per-image running index of plates found

        for r in results:  # one Results object per image
            boxes_xyxy = r.boxes.xyxy.cpu().numpy()
            scores     = r.boxes.conf.cpu().numpy()   # <<< FIX: 'conf' instead of 'class_conf'
            class_ids  = r.boxes.cls.cpu().numpy().astype(int)

            for (x1, y1, x2, y2), score, cls_id in zip(boxes_xyxy, scores, class_ids):
                # keep ONLY license plate detections
                if model.names[cls_id] != plate_class_name:
                    continue

                # crop the detected plate
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                crop = img[y1:y2, x1:x2]
                if crop.size == 0:
                    continue

                # resize width to STD_WIDTH, keep aspect ratio
                h, w = crop.shape[:2]
                new_h = max(1, int(h * STD_WIDTH / max(1, w)))
                crop = cv2.resize(crop, (STD_WIDTH, new_h), interpolation=cv2.INTER_CUBIC)

                # trim edges
                h_after, w_after = crop.shape[:2]
                top_px    = int(TOP_TRIM_FRAC    * h_after)
                bottom_px = int(BOTTOM_TRIM_FRAC * h_after)
                left_px   = int(LEFT_TRIM_FRAC   * w_after)
                right_px  = int(RIGHT_TRIM_FRAC  * w_after)

                crop = crop[
                    top_px : max(h_after - bottom_px, top_px + 1),
                    left_px: max(w_after  - right_px, left_px + 1)
                ]

                # file names
               # stem = Path(fname).stem
               # crop_name = f"{stem}_plate{plate_idx}.jpg"
                #crop_path = os.path.join(save_root, crop_name)

                # save raw crop
                cv2.imwrite(crop_path, crop)

                # preprocess for OCR
                proc = preprocess_plate(crop)
                proc_name = f"{stem}_plate{plate_idx}_prep.jpg"
                proc_path = os.path.join(save_root, proc_name)
                cv2.imwrite(proc_path, proc)

                # run OCR (LicensePlateRecognizer)
                # reader.run(...) may return a string or list; normalize to string
                plate_text = reader.run(proc)
                if isinstance(plate_text, (list, tuple)) and len(plate_text) > 0:
                    plate_text = plate_text[0]
                if plate_text is None:
                    plate_text = ""

                plate_text = str(plate_text).upper().replace(" ", "")

                # collect result row
                ocr_results.append({
                    "image name": fname,
                    "plate_idx" : plate_idx,
                    "plate_text": plate_text,
                    "score"     : float(score)
                })

                plate_idx += 1

        # (optional) save the full image as a .jpg to mirror input structure
        out_full = os.path.join(output_dir, os.path.splitext(rel)[0] + ".jpg")
        os.makedirs(os.path.dirname(out_full), exist_ok=True)
        cv2.imwrite(out_full, img)

        processed += 1
        if i % 10 == 0 or i == len(all_files):
            print(f"Processed {i}/{len(all_files)}")

    except Exception as e:
        failed.append((img_path, repr(e)))
        print(f"[Error] {img_path} -> {e}")
        traceback.print_exc()

print(f"\n✅ Done. Images processed: {processed}. Failures: {len(failed)}")
if failed:
    print("First few failures:")
    for p, err in failed[:5]:
        print(" -", p, "->", err)


Found 310 image(s).
Preview: ['IMG_2363.HEIC', 'IMG_2364.HEIC', 'IMG_2365.HEIC', 'IMG_2366.HEIC', 'IMG_2367.HEIC']

0: 480x640 1 License_Plate, 36.1ms
Speed: 5.9ms preprocess, 36.1ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 36.5ms
Speed: 5.1ms preprocess, 36.5ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 36.4ms
Speed: 5.7ms preprocess, 36.4ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 36.3ms
Speed: 6.2ms preprocess, 36.3ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 36.5ms
Speed: 5.4ms preprocess, 36.5ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 37.3ms
Speed: 5.8ms preprocess, 37.3ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 36.4ms
Speed: 6.2ms preprocess, 36.4ms inference, 3.9ms po

In [ ]:
# === Save OCR results to CSV ===
import os, csv
from pathlib import Path
from datetime import datetime

# 1) Where to save
#csv_path = os.path.join(output_dir, "plate_ocr_results.csv")

# 2) Make sure we actually have results
if not ocr_results:
    print("No OCR results to save (ocr_results is empty).")
else:
    # 3) Normalize records (ensure keys exist)
    normalized = []
    for r in ocr_results:
        normalized.append({
            "image_path": r.get("image_path", ""),
            "image_name": r.get("image name", r.get("image_name", "")),
            "plate_idx":  r.get("plate_idx", ""),
            "plate_text": r.get("plate_text", ""),
            "score":      r.get("score", ""),
        })

    # 4) Optional cleanup: drop empty plate_text rows and de‑dupe
    cleaned = []
    seen = set()
    for r in normalized:
        # keep only non-empty plate_text
        if not str(r["plate_text"]).strip():
            continue
        key = (r["image_path"], r["plate_idx"], r["plate_text"])
        if key in seen:
            continue
        seen.add(key)
        cleaned.append(r)

    # If everything got filtered, fall back to raw
    rows_to_write = cleaned if cleaned else normalized

    # 5) Sort for readability
    rows_to_write.sort(key=lambda x: (x["image_path"], x["plate_idx"] if str(x["plate_idx"]).isdigit() else 0))

    # 6) Write CSV (no extra deps)
    fieldnames = ["image_path", "image_name", "plate_idx", "plate_text", "score"]
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows_to_write)

    print(f"✅ Saved {len(rows_to_write)} rows to:\n{csv_path}")

    # 7) Quick preview
    for row in rows_to_write[:5]:
        print(row)


✅ Saved 312 rows to:
C:\Users\ajlee\Documents\LicensePlateProject\Processed Images Output\plate_ocr_results.csv
{'image_path': '', 'image_name': 'IMG_2363.HEIC', 'plate_idx': 0, 'plate_text': '9SIH263__', 'score': 0.8644153475761414}
{'image_path': '', 'image_name': 'IMG_2364.HEIC', 'plate_idx': 0, 'plate_text': 'ATOR100__', 'score': 0.857937216758728}
{'image_path': '', 'image_name': 'IMG_2365.HEIC', 'plate_idx': 0, 'plate_text': '9CCJ747__', 'score': 0.8899105191230774}
{'image_path': '', 'image_name': 'IMG_2366.HEIC', 'plate_idx': 0, 'plate_text': 'ARJ507A__', 'score': 0.79677414894104}
{'image_path': '', 'image_name': 'IMG_2367.HEIC', 'plate_idx': 0, 'plate_text': 'BIZN206__', 'score': 0.8440229296684265}
